# GCP Pub/Sub

O Pub/Sub é um serviço de mensagens assíncrono e escalável.
Nesse notebook iremos explorar os conceitos básicos de tópico, assinaturas, esquemas e retenção.

Os dados utilizados estão disponíveis no arquivo `aula-pdm-dados.zip`.

Referências: [documentação oficial](https://cloud.google.com/pubsub/docs/overview)

## Configurações

#### git clone

Caso esteja utilizando o dataproc, clone o repositório com os dados:

```bash
cd /home/dataproc
git clone https://github.com/robertogyn19/aula-pdm-pubsub.git
cd aula-pdm-pubsub
```

#### Obtenção dos dados

```bash
mkdir dados
cd dados

<TODO>
```

#### Versão do pubsub

Antes de iniciar, caso esteja utilizando o dataproc, verifique qual versão da lib python.
Isso pode ser feito através do terminal rodando o comando abaixo:

```bash
pip list | grep google-cloud-pubsub
google-cloud-pubsub               2.18.4
```

Caso a versão seja anterior a `2.27.0`, precisamos atualizá-la, faça isso rodando o comando abaixo:

```bash
pip install --upgrade google-cloud-pubsub

pip list | grep google-cloud-pubsub
google-cloud-pubsub               2.31.1
```

## 1. Tópicos

O primeiro exemplo que vamos ver é o mesmo do [tutorial inicial](https://cloud.google.com/python/docs/reference/pubsub/2.27.1) do cliente Python do Pub/Sub, apenas com alguns comentários para facilitar a compreensão.

In [ ]:
from google.cloud import pubsub_v1
from google.api_core.exceptions import AlreadyExists
from google.pubsub_v1 import BigQueryConfig, DeadLetterPolicy, Topic, Subscription
from google.protobuf import duration_pb2, field_mask_pb2, timestamp_pb2

In [ ]:
import google.auth

_, project_id = google.auth.default()
print(project_id)

In [ ]:
# Criação do cliente de publicação, com ele é possível criar tópicos e publicar mensagens
publisher = pubsub_v1.PublisherClient()

# Os nomes dos tópicos seguem o formato projects/<nome-do-projeto>/topics/<nome-do-tópico>
# o código abaixo configura esse nome
topic_name = "projects/{project_id}/topics/{topic}".format(
    project_id=project_id,
    topic="aula-pdm-primeiro-topico",
)
topic_name

In [ ]:
# Criação do tópico.
topico_obj = Topic({
    "name": topic_name,
    "message_retention_duration": "600s"  # 10 minutos
})

try:
    publisher.create_topic(request=topico_obj)
except AlreadyExists:
    print(f"O tópico '{topic_name}' já existe")

In [ ]:
# Publicação de algumas mensagens
for i in range(5):
    future = publisher.publish(topic_name, f"Minha mensagem {i}!".encode("utf-8"), versao="python3.11", ufg="aula-pdm")
    # A função que publica mensagens retorna um Future, pois é uma operação assíncrona.
    # Usamos a função result() para aguardar a resposta desse future
    print(f"{i} - {future.result()}")

## 2. Assinaturas ou subscriptions

Agora que publicamos nossa primeira mensagem, precisamos criar uma assinatura no tópico para consumir a mensagem.

In [ ]:
# Configuramos o nome da assinatura para o formato esperado assim como fizemos com o tópico
subscription_name = "projects/{project_id}/subscriptions/{sub}".format(
    project_id=project_id,
    sub="aula-pdm-minha-primeira-assinatura",
)
subscription_name

In [ ]:
# A função abaixo será responsável por receber a mensagem 
def callback(message):
    print(f"data: {message.data.decode('utf-8')} | attributes: {message.attributes}")
    # Essa chamada do ack é como o Pub/Sub controla quais mensagens foram processadas
    message.ack()

In [ ]:
# Criação do subscriber, com ele é possível criar assinaturas e "se inscrever" em tópicos
subscriber = pubsub_v1.SubscriberClient()

In [ ]:
# Criação da assinatura.
try:
    subscriber.create_subscription(
        name=subscription_name, topic=topic_name
    )
except AlreadyExists:
    print(f"A assinatura {subscription_name} já existe")

In [ ]:
# A chamada abaixo registra a assinatura com a função de callback
# Dessa forma, toda mensagem que chegar no tópico da assinatura, executará o código cadastrado
future = subscriber.subscribe(subscription_name, callback)

In [ ]:
# Assim como na publicação, a função result é para aguardar algo
# Ao executar o código abaixo, será que vai retornar algo?
# E por quanto tempo devemos esperar por novas mensagens?
try:
    future.result()
except KeyboardInterrupt:
    future.cancel()

In [ ]:
from datetime import datetime, timedelta, timezone

# 4) Fazer SEEK para 10 minutos atrás
#   (isso reposiciona o ponteiro da subscription para reentregar mensagens
#    publicadas desde esse instante, respeitando a retenção configurada)
dez_min_atras = datetime.now(timezone.utc) - timedelta(minutes=10)
ts = timestamp_pb2.Timestamp()
ts.FromDatetime(dez_min_atras)

subscriber.seek(request={"subscription": subscription_name, "time": ts})
print(f"Seek realizado para: {dez_min_atras.isoformat()}")

In [ ]:
# Agora vamos tentar novamente nos inscrever na assinatura
future = subscriber.subscribe(subscription_name, callback)

In [ ]:
# E agora, será que vai imprimir as mensagens?
try:
    future.result()
except KeyboardInterrupt:
    future.cancel()

## 3. Assinatura do BigQuery

Agora que já rodamos alguns códigos simples, vamos ver como podemos criar uma assinatura para ler os dados de um tópico de forma automática e inserir em uma tabela do BigQuery.

Essa seção foi baseada na [documentação oficial](https://cloud.google.com/pubsub/docs/create-bigquery-subscription?hl=pt-br).

Antes de criar a assinatura e inserir os dados, precisamos preparar o BigQuery.
Vamos realizar três procedimentos:
1. Criar um dataset no BigQuery, o dataset é onde as tabelas ficam agrupadas, como o esquema em bancos relacionais.
2. Criar a tabela no dataset do BigQuery.
3. Ajustar a permissão do Pub/Sub para conseguir enviar os dados.

Para criação do dataset e da tabela, vamos executar o comando a seguir no [BigQuery Studio](https://console.cloud.google.com/bigquery).

Clique no link `Consulta SQL` e cole o código abaixo.

![bigquery-studio](imagens-pubsub/img1-bq-studio.png)

```sql
-- Criação do dataset
CREATE SCHEMA IF NOT EXISTS aula_pdm;

-- Criação da tabela
CREATE TABLE IF NOT EXISTS `aula_pdm.anuncios` (
  site                            STRING,
  id_link                         INT64,
  anuncio_id                      INT64,
  descricao                       STRING,
  titulo                          STRING,
  endereco_completo               STRING,
  cep                             STRING,
  andar                           STRING,
  area_total                      NUMERIC,
  area_construida                 NUMERIC,
  area_util                       NUMERIC,
  qtd_quartos                     INT64,
  qtd_suites                      INT64,
  qtd_banheiros                   INT64,
  qtd_vagas_garagem               INT64,
  preco                           NUMERIC,
  iptu                            NUMERIC,
  coordenadas                     STRING,
  link_google_maps                STRING,
  print                           STRING,
  data_cadastro                   DATETIME,
  estado_do_imovel                STRING,
  status_do_anuncio               STRING,
  id_anuncio_plataforma           INT64,
  id_anuncio_externo              INT64,
  id                              INT64,
  data_atualizacao                DATETIME,
  deletado                        STRING,
  data_delecao                    DATETIME,
  coleta                          STRING,
  data_criacao_anuncio_plataforma DATETIME,
  caracteristicas                 STRING,
  imagens                         STRING
)
PARTITION BY DATE(data_cadastro)
;
```

Ao inserir o código acima, clique no botão `Executar` ou `Run`, ao finalizar, irá aparecer o status de cada comando executado.
Abaixo tem um exemplo de saída.

![bigquery-output](imagens-pubsub/img-bq-create-v2.png)

## 4. Integração entre BigQuery e Pub/Sub

Primeiro vamos criar o tópico que irá receber os dados de clientes.
O código é praticamente igual ao anterior.

In [ ]:
topico_anuncios = "projects/{project_id}/topics/{topic}".format(
    project_id=project_id,
    topic='aula-pdm-anuncios',
)
topico_anuncios

In [ ]:
publisher = pubsub_v1.PublisherClient()

In [ ]:
topico_obj = Topic({
    "name": topico_anuncios,
    "message_retention_duration": "259200s"  # 3 dias
})
try:
    publisher.create_topic(request=topico_obj)
except AlreadyExists:
    print(f"O tópico '{topico_anuncios}' já existe")

### 4.1) Permissões do Pub/Sub para o BigQuery

Para o Pub/Sub conseguir enviar os dados para o BigQuery, precisamos ajustar as permissões.
Execute o código abaixo de dentro do Cloud Shell ou do terminal da sua máquina local.

```shell
# Obtém o ID do projeto
export PROJECT_ID=$(gcloud config get project)

# Obtém o número do projeto
export PROJECT_NUMBER=$(gcloud projects describe "$PROJECT_ID" --format='value(projectNumber)')

# Conta de serviço do Pub/Sub
export SA=$(echo "service-$PROJECT_NUMBER@gcp-sa-pubsub.iam.gserviceaccount.com")

# Concede as permissões necessárias
gcloud projects add-iam-policy-binding "$PROJECT_ID" \
  --member="serviceAccount:$SA" \
  --role="roles/bigquery.dataEditor"

gcloud projects add-iam-policy-binding "$PROJECT_ID" \
  --member="serviceAccount:$SA" \
  --role="roles/storage.admin"
```

### 4.2) Criação de assinaturas pela interface gráfica

Antes de criar a assinatura através da interface gráfica, vamos criar um tópico para DLQ para ser utilizado na assinatura.

In [ ]:
# Configuração do nome do tópico para DLQ
topico_dlq = "projects/{project_id}/topics/{topic}".format(
    project_id=project_id,
    topic='aula-pdm-anuncios-bq-dlq',
)
topico_dlq
try:
    # Criação do tópico de DLQ
    publisher.create_topic(name=topico_dlq)
except AlreadyExists:
    print(f"O tópico '{topico_dlq}' já existe")

![new-subscription](imagens-pubsub/img3-new-sub.png)

![create-subscription](imagens-pubsub/img-v2-sub-bq1.png)

![create-subscription-2](imagens-pubsub/img-v2-sub-bq2.png)

![create-subscription-3](imagens-pubsub/img-v2-sub-bq3.png)

Após a criação, deverá aparecer a tela da assinatura como essa imagem a seguir.
Clique no botão `Conceder o papel de editor` ou `Grant Editor Role` para dar permissão para a conta de serviço do Pub/Sub.

![subscription-created](imagens-pubsub/img5-sub-dlq-access-pt.png)

### 4.2) Criação de assinaturas através de código python

A seguir temos o código para criar uma assinatura do BigQuery assim como fizemos pela interface gráfica.

In [ ]:
# Configuração do nome da assinatura
assinatura_anuncios_bq = "projects/{project_id}/subscriptions/{sub}".format(
    project_id=project_id,
    sub="aula-pdm-anuncios-bq-python",
)
assinatura_anuncios_bq

In [ ]:
# Configuração do nome do tópico para DLQ
topico_dlq = "projects/{project_id}/topics/{topic}".format(
    project_id=project_id,
    topic='aula-pdm-anuncios-bq-dlq',
)
topico_dlq

In [ ]:
try:
    # Criação do tópico de DLQ
    publisher.create_topic(name=topico_dlq)
except AlreadyExists:
    print(f"O tópico '{topico_dlq}' já existe")

In [ ]:
# Configuração da assinatura
anuncios_assinatura_bq = Subscription({
    "name": assinatura_anuncios_bq,
    "topic": topico_anuncios,
    "bigquery_config": BigQueryConfig({
        "table": f"{project_id}.aula_pdm.anuncios",  # nome da tabela no BigQuery
        "use_table_schema": True,  # flag indicando para utilizar o esquema da tabela
        "drop_unknown_fields": True  # flag indicando para descartar os campos desconhecidos
    }),
    "dead_letter_policy": DeadLetterPolicy({
        "dead_letter_topic": topico_dlq
    })
})

In [ ]:
try:
    subscriber.create_subscription(
        request=anuncios_assinatura_bq
    )
except AlreadyExists:
    print(f"A assinatura {assinatura_anuncios_bq} já existe")

### Exercício prático: Como seria uma assinatura para utilizar o esquema do tópico?

O Pub/Sub tem uma funcionalidade para criar esquemas e depois os atribuir a um ou mais tópicos.
Podemos definir os esquemas em dois formatos, [Avro Schema](https://avro.apache.org/docs/1.11.1/specification/) ou Protobuf.
Abaixo temos o esquema para algumas colunas da tabela anúncios:

```json
{
 "type" : "record",
 "name" : "Avro",
 "fields" : [
    {
      "name": "site",
      "type": "string"
    },
    {
      "name": "id_link",
      "type": "int"
    },
    {
      "name": "anuncio_id",
      "type": "int"
    },
    {
      "name": "descricao",
      "type": "string"
    },
    {
      "name": "titulo",
      "type": "string"
    },
    {
      "name": "area_total",
      "type": "float"
    },
    {
      "name": "qtd_quartos",
      "type": "int"
    },
    {
      "name": "preco",
      "type": "float"
    },
    {
      "name": "data_cadastro",
      "type": "int",
      "logicalType": "date"
    }
  ]
}
```

Tente criar o esquema acima e depois crie uma assinatura que envia os dados para o BigQuery utilizando o esquema do tópico.

Execute a consulta abaixo para criar uma tabela no BigQuery.
```sql
CREATE TABLE aula_pdm.anuncios_raw(
    data JSON
);
```

## 5. Envio de mensagens para o tópico

Vamos abrir o notebook `publicacao-anuncios.ipynb` e executar o código para enviar os dados para o tópico `aula-pdm-anuncios`.

## 6. Assinatura do Google Cloud Storage (GCS)

Existe um tipo de assinatura que envia os dados para o GCS em formato Avro. Essa funcionalidade pode ser interessante quando você tem um grande volume de dados ou quer economizar ou não precisa acessar os dados pelo BigQuery.

Essa seção foi baseada na [documentação oficial](https://cloud.google.com/pubsub/docs/create-cloudstorage-subscription?hl=pt-br).

### 6.1. Criação do bucket no GCS

Para criar uma assinatura do GCS é necessário criar um bucket e ajustar as permissões do mesmo.
Vamos criar um bucket pela [interface da GCP](https://console.cloud.google.com/storage/browser).

![create-bucket](imagens-pubsub/img6-create-bucket.png)

### 6.2. Criação da assinatura pela interface da GCP

Agora vamos criar a assinatura no Pub/Sub através da interface gráfica.

![create-subscription-gcs](imagens-pubsub/img8-sub-gcs.png)

![create-subscription-gcs-2](imagens-pubsub/img9-sub-gcs-file.png)